# Search EarthData's Common Metadata Repository by a bbox to get local ADAPT paths of ATL08 granules 
To find the ATL08 data you need to run through extraction and filtering, use the list returned from this noteboook 

Paul Montesano, Caleb Spadlin  
September 2023

### To run `do_extract_atl08_v005.py` like this:
```[pmontesa@adaptlogin101 ~]$ pdsh -g forest do_extract_filter_atl08.sh \"2018 2019 2020 2021 2022 2023\" /explore/nobackup/people/pmontesa/userfs02/data/icesat2/atl08.006/senegal_20m/list_atl08.006_senegal senegal_20m /explore/nobackup/people/pmontesa/userfs02/data/icesat2/atl08.006```


In [1]:
import datetime
import json
import warnings

import certifi
import urllib3
from urllib.parse import urlencode

from pprint import pprint

import pandas as pd
import numpy as np

import sys
#sys.path.append('/home/pmontesa/code/geoscitools')
#import atl08lib
#import maplib
import pandas as pd
import geopandas as gpd
import csv

/tmp/ipykernel_1557859/2774716887.py:19: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd


In [11]:
# -----------------------------------------------------------------------------
# class CmrProcess
#
# @author: Caleb Spradlin, caleb.s.spradlin@nasa.gov
# @version: 12.30.2021
#
# https://cmr.earthdata.nasa.gov/search/
# https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html
# -----------------------------------------------------------------------------
class CmrProcess(object):

    CMR_BASE_URL = 'https://cmr.earthdata.nasa.gov' +\
        '/search/granules.umm_json_v1_4?'

    # Range for valid lon/lat
    LATITUDE_RANGE = (-90, 90)
    LONGITUDE_RANGE = (-180, 180)

    # -------------------------------------------------------------------------
    # __init__
    # -------------------------------------------------------------------------
    def __init__(self,
                 mission,
                 dateTime,
                 lonLat=None,
                 error=False,
                 dayNightFlag='',
                 pageSize=150,
                 maxPages=50):

        self._error = error
        self._dateTime = dateTime
        self._mission = mission
        self._pageSize = pageSize
        self._maxPages = maxPages
        
        self._lonLat = lonLat
        self._dayNightFlag = dayNightFlag

    # -------------------------------------------------------------------------
    # run()
    #
    # Given a set of parameters on init (time, location, mission), search for
    # the most relevant file. This uses CMR to search metadata for
    # relevant matches.
    # -------------------------------------------------------------------------
    def run(self):
        print('Starting query')
        outout = set()
        for i in range(self._maxPages):
            
            d, e = self._cmrQuery(pageNum=i+1)
            
            if e and i > 1:
                return sorted(list(outout))
            
            if not e:
                print('Results found on page: {}'.format(i+1))
                out = [r['file_url'] for r in d.values()]
                outout.update(out)
                
        outout = sorted(list(outout))
        return outout
        

    # -------------------------------------------------------------------------
    # cmrQuery()
    #
    # Search the Common Metadata Repository(CMR) for a file that
    # is a temporal and spatial match.
    # -------------------------------------------------------------------------
    def _cmrQuery(self, pageNum=1):

        requestDictionary = self._buildRequest(pageNum=pageNum)
        totalHits, resultDictionary = self._sendRequest(requestDictionary)

        if self._error:
            return None, self._error

        if totalHits <= 0:
            print('No hits on page number: {}, ending search.'.format(pageNum))
            #warnings.warn(msg)
            return None, True

        resultDictionaryProcessed = self._processRequest(resultDictionary)
        return resultDictionaryProcessed, self._error

    # -------------------------------------------------------------------------
    # buildRequest()
    #
    # Build a dictionary based off of parameters given on init.
    # This dictionary will be used to encode the http request to search
    # CMR.
    # -------------------------------------------------------------------------
    def _buildRequest(self, pageNum=1):
        requestDict = dict()
        requestDict['page_num'] = pageNum
        requestDict['page_size'] = self._pageSize
        requestDict['concept_id'] = self._mission
        requestDict['bounding_box'] = self._lonLat
        requestDict['day_night_flag'] = self._dayNightFlag
        requestDict['temporal'] = self._dateTime
        return requestDict

    # -------------------------------------------------------------------------
    # _sendRequest
    #
    # Send an http request to the CMR server.
    # Decode data and count number of hits from request.
    # -------------------------------------------------------------------------
    def _sendRequest(self, requestDictionary):
        with urllib3.PoolManager(cert_reqs='CERT_REQUIRED',
                                 ca_certs=certifi.where()) as httpPoolManager:
            encodedParameters = urlencode(requestDictionary, doseq=True)
            requestUrl = self.CMR_BASE_URL + encodedParameters
            try:
                requestResultPackage = httpPoolManager.request('GET',
                                                               requestUrl)
            except urllib3.exceptions.MaxRetryError:
                self._error = True
                return 0, None

            requestResultData = json.loads(
                requestResultPackage.data.decode('utf-8'))
            status = int(requestResultPackage.status)

            if not status == 400:
                totalHits = len(requestResultData['items'])
                return totalHits, requestResultData

            else:
                msg = 'CMR Query: Client or server error: ' + \
                    'Status: {}, Request URL: {}, Params: {}'.format(
                        str(status), requestUrl, encodedParameters)
                warnings.warn(msg)
                return 0, None

    # -------------------------------------------------------------------------
    # _processRequest
    #
    # For each result in the CMR query, unpackage relevant information to
    # a dictionary. While doing so set flags if data is not desirable (too
    # close to edge of dataset).
    #
    #  REVIEW: Make the hard-coded names class constants? There are a lot...
    # -------------------------------------------------------------------------
    def _processRequest(self, resultDict):

        resultDictProcessed = dict()

        for hit in resultDict['items']:

            fileName = hit['umm']['RelatedUrls'][0]['URL'].split(
                '/')[-1]

            # ---
            # These are hardcoded here because the only time these names will
            # ever change is if we changed which format of metadata we wanted
            # the CMR results back in.
            #
            # These could be placed as class constants in the future.
            # ---
            fileUrl = hit['umm']['RelatedUrls'][0]['URL']
            temporalRange = hit['umm']['TemporalExtent']['RangeDateTime']
            dayNight = hit['umm']['DataGranule']['DayNightFlag']

 
            spatialExtent = hit['umm']['SpatialExten' +
                                          't']['HorizontalSpatialDom' +
                                               'ain']

            key = fileName

            resultDictProcessed[key] = {
                'file_name': fileName,
                'file_url': fileUrl,
                'temporal_range': temporalRange,
                'spatial_extent': spatialExtent,
                'day_night_flag': dayNight}

        return resultDictProcessed

#### Boreal NA

In [12]:
search_dict = {
    'site_name': 'Nome Field Trip',
    'bbox': [-165.47607,64.83343,-164.70703,65.07524], # 64.83343,-165.47607 # 65.07524,-164.70703
    'minmonth': "07",
    'maxmonth': "08",
    'years_list': [2019, 2020]
}

## Choose a site and create subdir

In [13]:
# Find this at https://search.earthdata.nasa.gov/
COLLECTIONCONCEPTID_DICT = {
                        'ATL08.003': "C2003772626-NSIDC_ECS",
                        'ATL08.005': "C2144424132-NSIDC_ECS",
                        'ATL08.006': "C2565090645-NSIDC_ECS",
                        'ATL03.006': 'C2559919423-NSIDC_ECS'
}

In [14]:
# This provides the ATLAS data product and version - also used for subdir in ADAPT/EXPLORE
ATL08_VERSION = 'ATL03.006'

In [15]:
# Page size: 150, number of results returned by page.
PAGESIZE = 150 
# Max page, number of pages to return before ending query.
MAXPAGE = 60
# Total max results will be PAGESIZE * MAXPAG

cmrP_list = []
for YEAR in search_dict['years_list']:
    
    cmrP = CmrProcess(mission = COLLECTIONCONCEPTID_DICT[ATL08_VERSION], 
                      dateTime=f"{YEAR}-{search_dict['minmonth']}-01T00:00:00Z,{YEAR}-{search_dict['maxmonth']}-30T23:59:59Z", 
                      lonLat = ','.join(str(e) for e in search_dict['bbox']),
                      pageSize=PAGESIZE,
                      maxPages=MAXPAGE)
    cmrP_list.append(cmrP)

In [16]:
resultList_year = [cmrP.run() for cmrP in cmrP_list]

Starting query
Results found on page: 1
No hits on page number: 2, ending search.
No hits on page number: 3, ending search.
Starting query
Results found on page: 1
No hits on page number: 2, ending search.
No hits on page number: 3, ending search.


In [10]:
resultList_year

[['https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.006/2019.07.12/ATL03_20190712131827_02210403_006_02.h5',
  'https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.006/2019.07.16/ATL03_20190716131008_02820403_006_02.h5',
  'https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.006/2019.07.26/ATL03_20190726011248_04270405_006_02.h5',
  'https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.006/2019.08.10/ATL03_20190810115440_06630403_006_02.h5',
  'https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.006/2019.08.14/ATL03_20190814114620_07240403_006_02.h5',
  'https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.006/2019.08.23/ATL03_20190823234858_08690405_006_02.h5'],
 ['https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.006/2020.07.09/ATL03_20200709195748_02210803_006_01.h5',
  'https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.006/2020.07.23/ATL03_20200723075207_04270805_006_01.h5',
  'https://n5eil01u.ecs.nsidc.org/DP9/ATLAS/ATL03.006/2020.08.07/ATL03_20200807183352_06630803_006_01.h5',
  'https://n5eil01u.ecs.nsidc.org/DP